In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('solar2019.csv')
df.head(100)
idx_df = df.set_index(['时间'])
idx_df.describe().T  #用于计算数值列的描述性统计信息，如计数、均值、标准差、最小值、最大值等
idx_df.head(10)
idx_df.isnull().sum()#DataFrame 中每列的缺失值数量。通过这个操作，您可以了解每列中缺失值的分布情况，并进行进一步的数据清洗或处理。
idx_df.corr()['实际发电功率(mw)'].sort_values(ascending=False)
print(idx_df.shape)
idx_df.info()
df1 = idx_df['实际发电功率(mw)']
df2 = idx_df.drop(['实际发电功率(mw)'],axis=1)

print(df1.shape)
from vmdpy import VMD
tau = 0.  # noise-tolerance (no strict fidelity enforcement)
DC = 0  # no DC part imposed
init = 1  # initialize omegas uniformly
tol = 1e-7
K=5
alpha=2000
u, u_hat, omega = VMD(df1, alpha, tau, K, DC, init, tol)
print(u)
u=u.T
print(u.shape)
print(u)
x = u[:,0]
print(x.shape)
# 将x = u[:,0]作为一个新的列添加到df2
df['imf1']=u[:,0]
df['imf2']=u[:,1]
df['imf3']=u[:,2]
df['imf4']=u[:,3]
df['imf5']=u[:,4]
idx_df.corr()['实际发电功率(mw)'].sort_values(ascending=False) #在这里环境的阈值和vmd的分量的阈值都定义为0.1
df2.shape
print(df2)


import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Bidirectional,GRU,Input, Conv1D,Multiply,Flatten,MaxPooling1D,concatenate,GlobalAveragePooling1D,Reshape
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout,Permute,Lambda,RepeatVector
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

#通道注意力机制
def SE_Block(input_tensor,ratio = 2):
    input_shape = K.int_shape(input_tensor)
    squeeze = tf.keras.layers.GlobalAveragePooling1D()(input_tensor)
    excitation = tf.keras.layers.Dense(units = input_shape[-1]//ratio, kernel_initializer='he_normal',activation='relu')(squeeze)
    excitation = tf.keras.layers.Dense(units = input_shape[-1],activation='sigmoid')(excitation)
    #excitation = tf.reshape(excitation, [-1, 1, input_shape[-1]])
    scale = tf.keras.layers.Multiply()([input_tensor, excitation])
    return scale


predictions_list = []  # 创建一个空列表用于存储预测结果

for i in range(5):
    x = u[:, i]
    df2['x'] = u[:, i]
    #   print(df2)
    y_train_set = df2.iloc[:24528, 7:8].values  # 目标点
    y_val_set = df2.iloc[24528:31536, 7:8].values  # 目标点
    # y_test_set = df2.iloc[31536:35040,7:8].values

    x_train_set = df2.iloc[:24528, :8].values  # 其它特征
    x_val_set = df2.iloc[24528:31536, :8].values
    x_test_set = df2.iloc[31536:35040, :8].values

    x_sc = MinMaxScaler()
    x_train2 = x_sc.fit_transform(x_train_set)
    x_val2 = x_sc.transform(x_val_set)
    x_test2 = x_sc.transform(x_test_set)
    y_sc = MinMaxScaler()
    y_train2 = y_sc.fit_transform(y_train_set)
    y_val2 = y_sc.transform(y_val_set)

    x_train = []
    y_train = []

    x_val = []
    y_val = []

    x_test = []

    # 利用for循环，
    for i in range(8, len(x_train2)):
        x_train.append(x_train2[i - 8:i, :])
        y_train.append(y_train2[i, :])

    # 对训练集进行打乱
    np.random.seed(7)
    np.random.shuffle(x_train)
    np.random.seed(7)
    np.random.shuffle(y_train)
    tf.random.set_seed(7)
    # 将训练集由list格式变为array格式
    x_train, y_train = np.array(x_train), np.array(y_train)

    y_train = np.reshape(y_train, (y_train.shape[0],))

    # 使x_train符合RNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]。

    # 利用for循环
    for i in range(8, len(x_val2)):
        x_val.append(x_val2[i - 8:i, :])
        y_val.append(y_val2[i, :])
    # 测试集变array并reshape为符合LSTM输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]
    x_val, y_val = np.array(x_val), np.array(y_val)
    y_val = np.reshape(y_val, (y_val.shape[0],))  # 标签为一维数组

    for i in range(8, len(x_test2)):
        x_test.append(x_test2[i - 8:i, :])

    # 测试集变array并reshape为符合LSTM输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]
    x_test = np.array(x_test)


    def double_CNN_GRU():
        visible1 = Input(shape=(8, 8))
        #     cnn1 = Conv1D(filters=8, kernel_size=1, activation='relu',padding='same')(visible1)   #output_length = seq_length - kernel_size + 1=24-3+1=22  (22,8)
        #     cnn1 = MaxPooling1D(pool_size=2)(cnn1)  #output_length = ceil(seq_length / pool_size)=22/2=11   (11,8)
        #     cnn2 = Conv1D(filters=16, kernel_size=3, activation='relu')(cnn1)  #11-3+1=9   (9,16)
        #     cnn2 = MaxPooling1D(pool_size=2)(cnn2)   # (9/2,16)=(4,16)
        #     cnn = Flatten()(cnn2)  #4*16=64
        #     cnn2_with_se = SE_Block(visible1, ratio=8)
        #     SENet = Flatten()(cnn2_with_se)

        #     cs = SE_Block(visible1,ratio = 2)
        cnn1 = Conv1D(filters=8, kernel_size=1, activation='relu', padding='same')(visible1)
        #     cnn1 = MaxPooling1D(pool_size=2)(cnn1)
        cs = SE_Block(cnn1, ratio=2)
        cnn2 = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(cs)  # 11-3+1=9   (9,16)
        cnn2 = MaxPooling1D(pool_size=2)(cnn2)  # (9/2,16)=(4,16)
        cnn = Flatten()(cnn2)

        visible2 = Input(shape=(8, 8))
        gru1 = Bidirectional(GRU(units=32, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.01)))(
            visible2)  # (24,32)
        gru2 = Bidirectional(GRU(units=16, return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(0.01)))(
            gru1)
        gru = Flatten()(gru2)

        merge = concatenate([cnn, gru])

        #     # 添加自注意力层
        #     self_attention_layer = SelfAttention(d_model=32)  # 假设自注意力层的输出维度为32
        #     attention = self_attention_layer(merge)
        # 用于将输入层的数据压成一维的数据，一般用再卷积层和全连接层之间
        attention_probs = Dense(96, activation='sigmoid', name='attention_vec')(merge)
        attention_mul = Multiply()([merge, attention_probs])
        F = Flatten()(attention_mul)

        dense = Dense(units=64, activation='relu')(F)
        output = Dense(units=1, activation='relu')(dense)
        model = Model(inputs=[visible1, visible2], outputs=output)
        adam = Adam(learning_rate=0.001)
        return model


    m = double_CNN_GRU()
    m.summary()
    m.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    hist = m.fit([x_train, x_train], y_train, epochs=100, batch_size=32, validation_data=([x_val, x_val], y_val),
                 verbose=1)

    y_pred = m.predict([x_test, x_test])

    # 对预测数据还原---从（0，1）反归一化到原始范围
    y_pred_original = y_sc.inverse_transform(y_pred)

    predictions_list.append(y_pred_original)

import numpy as np

predictions_array = np.array(predictions_list)

l = predictions_array.T

print(l.shape)
reshaped_l = l.reshape((3496, 5))

print(reshaped_l.shape)

print(reshaped_l)

# 将第一列到第五列的值相加，并将结果放在第六列
VMDSUM = np.sum(reshaped_l[:, [0,1,2,3,4]], axis=1)

print("处理后的数组:")
print(VMDSUM)

df4 = idx_df
y_test_set = df4.iloc[31536:35040,7:8].values

print(y_test_set)
y_test = []

for i in range(8, len(y_test_set)):
    y_test.append(y_test_set[i, :])
# 测试集变array并reshape为符合LSTM输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]
y_test = np.array(y_test)
y_test = np.reshape(y_test,(y_test.shape[0],)) #标签为一维数组
print(y_test.shape)

y_pred_original = VMDSUM
y_test_original = y_test


# 计算均方误差 (MSE) 和均方根误差 (RMSE)
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score,explained_variance_score


# 计算NRMSE
def normalized_root_mean_squared_error(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    range_y = np.max(y_true) - np.min(y_true)
    return rmse / range_y

mse = mean_squared_error(y_test_original, y_pred_original)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_original, y_pred_original)
# 计算 MBE
mbe = np.mean(y_test_original - y_pred_original)
nrmse = normalized_root_mean_squared_error(y_test_original, y_pred_original)
r2 = r2_score(y_test_original, y_pred_original)
evs = explained_variance_score(y_test_original, y_pred_original)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("Mean Bias Error(MBE):",mbe)
print("NRMSE:", nrmse)
print("Explained Variance Score (EVS):", evs)
print("R\xb2 Score:", r2)

(35040, 8)
<class 'pandas.core.frame.DataFrame'>
Index: 35040 entries, 2019-01-01 00:00:00 to 2019-12-31 23:45:00
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   组件温度(℃)     35040 non-null  float64
 1   温度(°)       35040 non-null  float64
 2   气压(hPa)     35040 non-null  float64
 3   湿度(%)       35040 non-null  float64
 4   总辐射(W/m2)   35040 non-null  float64
 5   直射辐射(W/m2)  35040 non-null  float64
 6   散射辐射(W/m2)  35040 non-null  float64
 7   实际发电功率(mw)  35040 non-null  float64
dtypes: float64(8)
memory usage: 2.4+ MB
(35040,)
[[ 6.68138428e+00  6.68258673e+00  6.68499730e+00 ...  1.21019030e+01
   1.21088950e+01  1.21122538e+01]
 [-1.14695053e+01 -1.14350936e+01 -1.13663572e+01 ... -1.81754613e+01
  -1.83692553e+01 -1.84665058e+01]
 [ 4.08477465e+00  4.08528636e+00  4.08501313e+00 ...  8.58558664e+00
   9.07293305e+00  9.31943689e+00]
 [ 7.45074008e-01  7.15429239e-01  6.56304161e-01 ... -2.66265671e+00
  -2.9788

767/767 [==============================] - 5s 7ms/step - loss: 1.0759e-05 - root_mean_squared_error: 0.0033 - val_loss: 1.0363e-05 - val_root_mean_squared_error: 0.0032
Epoch 49/100
767/767 [==============================] - 5s 7ms/step - loss: 9.1226e-06 - root_mean_squared_error: 0.0030 - val_loss: 1.2820e-05 - val_root_mean_squared_error: 0.0036
Epoch 50/100
767/767 [==============================] - 5s 7ms/step - loss: 1.0323e-05 - root_mean_squared_error: 0.0032 - val_loss: 1.0508e-05 - val_root_mean_squared_error: 0.0032
Epoch 51/100
767/767 [==============================] - 5s 7ms/step - loss: 1.0547e-05 - root_mean_squared_error: 0.0032 - val_loss: 1.6196e-05 - val_root_mean_squared_error: 0.0040
Epoch 52/100
767/767 [==============================] - 5s 7ms/step - loss: 9.6256e-06 - root_mean_squared_error: 0.0031 - val_loss: 8.2598e-06 - val_root_mean_squared_error: 0.0029
Epoch 53/100
767/767 [==============================] - 5s 7ms/step - loss: 9.7290e-06 - root_mean_squa

767/767 [==============================] - 5s 7ms/step - loss: 2.5119e-05 - root_mean_squared_error: 0.0050 - val_loss: 7.0349e-05 - val_root_mean_squared_error: 0.0084
Epoch 14/100
767/767 [==============================] - 5s 7ms/step - loss: 2.6190e-05 - root_mean_squared_error: 0.0051 - val_loss: 1.0376e-04 - val_root_mean_squared_error: 0.0102
Epoch 15/100
767/767 [==============================] - 5s 7ms/step - loss: 2.0291e-05 - root_mean_squared_error: 0.0045 - val_loss: 4.2145e-05 - val_root_mean_squared_error: 0.0065
Epoch 16/100
767/767 [==============================] - 5s 7ms/step - loss: 1.9220e-05 - root_mean_squared_error: 0.0044 - val_loss: 3.7225e-05 - val_root_mean_squared_error: 0.0061
Epoch 17/100
767/767 [==============================] - 5s 7ms/step - loss: 2.0020e-05 - root_mean_squared_error: 0.0045 - val_loss: 1.1129e-04 - val_root_mean_squared_error: 0.0105
Epoch 18/100
767/767 [==============================] - 5s 7ms/step - loss: 2.3245e-05 - root_mean_squa

767/767 [==============================] - 10s 7ms/step - loss: 0.4173 - root_mean_squared_error: 0.1077 - val_loss: 1.8042e-04 - val_root_mean_squared_error: 0.0132
Epoch 2/100
767/767 [==============================] - 5s 7ms/step - loss: 1.0830e-04 - root_mean_squared_error: 0.0103 - val_loss: 1.2819e-04 - val_root_mean_squared_error: 0.0113
Epoch 3/100
767/767 [==============================] - 5s 7ms/step - loss: 7.7605e-05 - root_mean_squared_error: 0.0088 - val_loss: 9.8682e-05 - val_root_mean_squared_error: 0.0099
Epoch 4/100
767/767 [==============================] - 5s 7ms/step - loss: 5.8444e-05 - root_mean_squared_error: 0.0076 - val_loss: 7.6976e-05 - val_root_mean_squared_error: 0.0088
Epoch 5/100
767/767 [==============================] - 5s 7ms/step - loss: 6.1223e-05 - root_mean_squared_error: 0.0078 - val_loss: 1.0343e-04 - val_root_mean_squared_error: 0.0102
Epoch 6/100
767/767 [==============================] - 5s 7ms/step - loss: 4.0919e-05 - root_mean_squared_erro

767/767 [==============================] - 5s 7ms/step - loss: 3.6851e-06 - root_mean_squared_error: 0.0019 - val_loss: 2.9686e-06 - val_root_mean_squared_error: 0.0017
Epoch 92/100
767/767 [==============================] - 5s 7ms/step - loss: 3.4817e-06 - root_mean_squared_error: 0.0019 - val_loss: 7.0332e-06 - val_root_mean_squared_error: 0.0026
Epoch 93/100
767/767 [==============================] - 5s 6ms/step - loss: 4.5283e-06 - root_mean_squared_error: 0.0021 - val_loss: 2.8983e-06 - val_root_mean_squared_error: 0.0017
Epoch 94/100
767/767 [==============================] - 5s 7ms/step - loss: 4.2545e-06 - root_mean_squared_error: 0.0020 - val_loss: 3.6852e-06 - val_root_mean_squared_error: 0.0019
Epoch 95/100
767/767 [==============================] - 5s 7ms/step - loss: 4.4378e-06 - root_mean_squared_error: 0.0021 - val_loss: 8.3017e-06 - val_root_mean_squared_error: 0.0029
Epoch 96/100
767/767 [==============================] - 6s 8ms/step - loss: 3.5965e-06 - root_mean_squa

767/767 [==============================] - 5s 7ms/step - loss: 1.5931e-05 - root_mean_squared_error: 0.0040 - val_loss: 2.7055e-05 - val_root_mean_squared_error: 0.0052
Epoch 57/100
767/767 [==============================] - 5s 7ms/step - loss: 1.6812e-05 - root_mean_squared_error: 0.0041 - val_loss: 2.1527e-05 - val_root_mean_squared_error: 0.0046
Epoch 58/100
767/767 [==============================] - 5s 7ms/step - loss: 1.3990e-05 - root_mean_squared_error: 0.0037 - val_loss: 2.0086e-05 - val_root_mean_squared_error: 0.0045
Epoch 59/100
767/767 [==============================] - 5s 7ms/step - loss: 1.5527e-05 - root_mean_squared_error: 0.0039 - val_loss: 3.4621e-05 - val_root_mean_squared_error: 0.0059
Epoch 60/100
767/767 [==============================] - 5s 7ms/step - loss: 1.5846e-05 - root_mean_squared_error: 0.0040 - val_loss: 5.3285e-05 - val_root_mean_squared_error: 0.0073
Epoch 61/100
767/767 [==============================] - 5s 7ms/step - loss: 1.7082e-05 - root_mean_squa

767/767 [==============================] - 11s 8ms/step - loss: 0.4075 - root_mean_squared_error: 0.0689 - val_loss: 4.5146e-04 - val_root_mean_squared_error: 0.0212
Epoch 2/100
767/767 [==============================] - 6s 7ms/step - loss: 3.2266e-04 - root_mean_squared_error: 0.0179 - val_loss: 4.1659e-04 - val_root_mean_squared_error: 0.0204
Epoch 3/100
767/767 [==============================] - 6s 8ms/step - loss: 2.2241e-04 - root_mean_squared_error: 0.0149 - val_loss: 2.0259e-04 - val_root_mean_squared_error: 0.0142
Epoch 4/100
767/767 [==============================] - 6s 8ms/step - loss: 1.5535e-04 - root_mean_squared_error: 0.0125 - val_loss: 1.3613e-04 - val_root_mean_squared_error: 0.0117
Epoch 5/100
767/767 [==============================] - 6s 7ms/step - loss: 1.2648e-04 - root_mean_squared_error: 0.0112 - val_loss: 1.9041e-04 - val_root_mean_squared_error: 0.0138
Epoch 6/100
767/767 [==============================] - 6s 7ms/step - loss: 9.6604e-05 - root_mean_squared_erro

767/767 [==============================] - 5s 7ms/step - loss: 1.1344e-05 - root_mean_squared_error: 0.0034 - val_loss: 7.7572e-06 - val_root_mean_squared_error: 0.0028
Epoch 92/100
767/767 [==============================] - 6s 7ms/step - loss: 1.0959e-05 - root_mean_squared_error: 0.0033 - val_loss: 9.3937e-06 - val_root_mean_squared_error: 0.0031
Epoch 93/100
767/767 [==============================] - 6s 7ms/step - loss: 9.8685e-06 - root_mean_squared_error: 0.0031 - val_loss: 1.5674e-05 - val_root_mean_squared_error: 0.0040
Epoch 94/100
767/767 [==============================] - 5s 7ms/step - loss: 1.2552e-05 - root_mean_squared_error: 0.0035 - val_loss: 8.3301e-06 - val_root_mean_squared_error: 0.0029
Epoch 95/100
767/767 [==============================] - 5s 7ms/step - loss: 1.1031e-05 - root_mean_squared_error: 0.0033 - val_loss: 8.7837e-06 - val_root_mean_squared_error: 0.0030
Epoch 96/100
767/767 [==============================] - 6s 7ms/step - loss: 1.0770e-05 - root_mean_squa

In [3]:
"""
Mean Squared Error (MSE): 2.5375927630864688
Root Mean Squared Error (RMSE): 1.5929823486424666
Mean Absolute Error (MAE): 0.9505564288293538
Mean Bias Error(MBE): -0.32050408159417654
NRMSE: 0.033702543862399736
Explained Variance Score (EVS): 0.987213849796436
R² Score: 0.986674424671573
"""

'\nMean Squared Error (MSE): 2.5375927630864688\nRoot Mean Squared Error (RMSE): 1.5929823486424666\nMean Absolute Error (MAE): 0.9505564288293538\nMean Bias Error(MBE): -0.32050408159417654\nNRMSE: 0.033702543862399736\nExplained Variance Score (EVS): 0.987213849796436\nR² Score: 0.986674424671573\n'